In [113]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import ClusterCentroids
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    confusion_matrix
)
from sklearn.preprocessing import StandardScaler

In [7]:
#начнём с загрузки данных
df = pd.read_csv('_train.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532823 entries, 0 to 532822
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gndr                  532823 non-null  int64  
 1   brth_yr               532823 non-null  int64  
 2   erly_pnsn_flg         532823 non-null  int64  
 3   prvs_npf              532823 non-null  int64  
 4   ch_north              532823 non-null  int64  
 5   gdp_growth_iY         532823 non-null  float64
 6   unemployment_rate_iY  532823 non-null  float64
 7   inflation_rate_iY     532823 non-null  float64
 8   interest_rate_iY      532823 non-null  float64
 9   est_pr_iY             532823 non-null  float64
 10  summ                  532823 non-null  float64
dtypes: float64(6), int64(5)
memory usage: 44.7 MB


In [9]:
df.erly_pnsn_flg.value_counts(normalize=True)

erly_pnsn_flg
0    0.963637
1    0.036363
Name: proportion, dtype: float64

Наблюдается довольно серьёзный дисбаланс классов. Требуется что-то с этим сделать.

In [11]:
# Разделение на признаки и целевую переменную
X = df.drop(columns=['erly_pnsn_flg'])  # Признаки
y = df['erly_pnsn_flg']  # Целевая переменна

Для начала используем различные методы, чтобы избавиться от дисбаланса классов. Это поможет обучить модель, которая будет иметь большую "насмотренность". Начнём с разделения на обучающую и тестовую выборку.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Посмотрим, получилось ли сохранить баланс классов.

In [15]:
# Проверка пропорций в обучающей и тестовой выборках
print("Train class distribution:\n", y_train.value_counts(normalize=True))
print("Test class distribution:\n", y_test.value_counts(normalize=True))

Train class distribution:
 erly_pnsn_flg
0    0.963637
1    0.036363
Name: proportion, dtype: float64
Test class distribution:
 erly_pnsn_flg
0    0.963637
1    0.036363
Name: proportion, dtype: float64


Сохранить исходный баланс классов получилось, поэтому можно рассмотреть различные методы для снижения дисбаланса классов.

Для начала используем метод SMOTE.
SMOTE — это метод повышения представительности редкого класса в задачах классификации с несбалансированными классами.

Он позволяет создать синтетические экземпляры для наименее представленного класса.

В отличие от простого увеличения данных (oversampling), SMOTE создаёт новые, уникальные экземпляры, что помогает модели лучше обобщать.
Новый экземпляр сохраняет некоторые свойства оригинальных данных, что позволяет модели извлекать больше информации.



In [21]:
smote = SMOTE(sampling_strategy=0.5, k_neighbors=5, random_state=42)

In [32]:
# Применение SMOTE к обучающей выборке
X_resampled_smote, y_resampled_smote = smote.fit_resample(X_train, y_train)

In [31]:
# Проверка нового распределения классов в обучающей выборке
print("Баланс классов после метода SMOTE:\n", y_train_balanced.value_counts(normalize=True))

Баланс классов после метода SMOTE:
 erly_pnsn_flg
0    0.666667
1    0.333333
Name: proportion, dtype: float64


Попробуем применить Random Undersmpling на SMOTE-выборку.
Нам это может помочь сохранить в классе большинства важную информацию, при условии что в классе меньшинства будет больше наблюдений.

In [33]:
# Random Undersampling, у нас уже есть SMOTE-выборка, её и передаём
rus = RandomUnderSampler(random_state=42)
X_resampled_smote_rus, y_resampled_smote_rus = rus.fit_resample(X_resampled_smote, y_resampled_smote)

Попробуем применить Cluster Centroids.
Метод кластеризации группирует наблюдения из класса большинства, и затем выбираются центры кластеров как репрезентативные примеры. 
Это позволяет сохранить важную информацию при уменьшении числа наблюдений.
Удаление наблюдений на основе кластеризации может помочь исключить шумные данные и выбросы, которые могут мешать обучению модели.

In [34]:
# Cluster Centroids
cc = ClusterCentroids(random_state=42)
X_resampled_cc, y_resampled_cc = cc.fit_resample(X_train, y_train)

SMOTEENN (SMOTE + Edited Nearest Neighbors).
SMOTE создает синтетические наблюдения для класса меньшинства, а Edited Nearest Neighbors очищает выборку, удаляя шумные наблюдения и выбросы. Это может привести к более чистым данным для обучения модели.

In [36]:
# SMOTEENN
smote_enn = SMOTEENN(random_state=42)
X_resampled_smote_enn, y_resampled_smote_enn = smote_enn.fit_resample(X_train, y_train)

Посмотрим какой баланс классов получился в результате применения этих методов

In [38]:
def class_proportions(y):
    counts = y.value_counts(normalize=True)
    return counts

In [39]:
print(f'SMOTE + Random Undersampling proportions:\n{class_proportions(y_resampled_smote_rus)}')

SMOTE + Random Undersampling proportions:
erly_pnsn_flg
0    0.5
1    0.5
Name: proportion, dtype: float64


In [40]:
print(f'Cluster Centroids proportions:\n{class_proportions(y_resampled_cc)}')

Cluster Centroids proportions:
erly_pnsn_flg
0    0.5
1    0.5
Name: proportion, dtype: float64


In [41]:
print(f'SMOTEENN proportions:\n{class_proportions(y_resampled_smote_enn)}')

SMOTEENN proportions:
erly_pnsn_flg
1    0.508487
0    0.491513
Name: proportion, dtype: float64


Теперь у нас на руках более сбалансированная выборка -> обучаем модели и смотрим на результаты.

Для модели предполагается сначала подобрать параметры для 3 моделей: LogisticRegression, RandomForestClassifier, XGBClassifier
После составить из них ансамбль Voting Classifier

Начнём с модели логистической регрессии.

Какие гиперпарметры мы будем учитывать:
1. C: Параметр регуляризации - высокие значения C приводят к меньшей регуляризации, а низкие — к более строгой регуляризации.
Влияние на модель: при слишком малом C модель может переобучиться (чрезмерно подстраиваться под обучающие данные),
а при слишком большом — недообучиться (не улавливать закономерности в данных).
2. penalty: Метод регуляризации -
L1 (Lasso): Устраняет менее важные признаки, что может привести к разреженной модели.
L2 (Ridge): Применяет штраф за большие коэффициенты, но не исключает признаки.
elasticnet: Сочетает L1 и L2, полезно, если много коррелирующих признаков.
3. solver: Алгоритм оптимизации
'lbfgs': Эффективен для больших наборов данных и поддерживает только L2.
'saga': Работает с большими наборами данных и поддерживает как L1, так и L2.
4.max_iter: Максимальное количество итераций.
Описание: Максимальное количество итераций для алгоритма оптимизации.
Влияние: Если max_iter слишком мал, алгоритм может не сходиться, особенно при больших данных или сложной модели.

Напишем функцию, куда сможем передавать гипермараметры, которые будут использоваться гридсерчем для проверки модели. Так будет удобнее.

In [59]:
def train_LogReg(X_resampled, y_resampled, C=[0.001, 0.01, 0.1, 1, 10, 100], 
                 penalty=['elasticnet'], l1_ratio=[0.1, 0.5, 0.9, 1], solver=['saga'], max_iter=[100, 200, 300, 400, 500],
                 scoring = 'roc_auc'):
    """
    Обучает модель логистической регрессии с использованием GridSearchCV для подбора гиперпараметров.

    Параметры:
    -----------
    X_resampled : array-like, shape (n_samples, n_features)
        Обучающие данные после балансировки (признаки).
        
    y_resampled : array-like, shape (n_samples,)
        Целевые переменные после балансировки (метки классов).
        
    C : list, optional (default=[0.001, 0.01, 0.1, 1, 10, 100])
        Гиперпараметр регуляризации. Определяет, насколько сильно будет применяться 
        регуляризация. Чем меньше значение, тем сильнее регуляризация.

    penalty : list, optional (default=['elasticnet'])
        Тип регуляризации, которая будет использоваться в модели. Доступные значения 
        включают 'l1', 'l2', и 'elasticnet'. При использовании 'elasticnet' требуется 
        указать l1_ratio.
        
    l1_ratio : list, optional (default=[0.1, 0.5, 0.9, 1])
        Соотношение между L1 и L2 регуляризацией. Значения от 0 (только L2) до 1 (только L1).
        Этот параметр игнорируется, если penalty не равен 'elasticnet'.
        
    solver : list, optional (default=['saga'])
        Алгоритм оптимизации, который будет использоваться для обучения модели. 
        Для 'elasticnet' необходимо использовать 'saga'.
        
    max_iter : list, optional (default=[100, 200, 300, 400, 500])
        Максимальное количество итераций для алгоритма оптимизации.
        
    scoring : str, optional (default='roc_auc')
        Метрика, используемая для оценки модели. Поддерживаемые значения включают 
        'roc_auc', 'accuracy', 'f1', и другие. Это значение будет использоваться 
        в качестве критерия для подбора гиперпараметров.

    Возвращаемое значение:
    -----------------------
    grid_search : GridSearchCV
        Объект GridSearchCV, содержащий информацию о лучших параметрах и оценках 
        модели.
    """
    
    if penalty == ['elasticnet']:
        param_grid = {
            'C': C,
            'penalty': penalty,  
            'l1_ratio': l1_ratio,  # Соотношение L1 и L2
            'solver': solver,  
            'max_iter': max_iter  
        }
    else:
        param_grid = {
        'C': C,
        'penalty': penalty,  
        'solver': solver,  
        'max_iter': max_iter  
    }
    
    # Создаем экземпляр GridSearch
    grid_search = GridSearchCV(LogisticRegression(random_state=42), param_grid, cv=5, scoring=scoring, n_jobs=-1)  # Используем n_jobs=-1 для параллельного выполнения
    
    # Обучение
    grid_search.fit(X_resampled, y_resampled)

    #возвращаем модель
    return grid_search

def print_bestparams(gr_sr): # Лучшие параметры
    print(f"Лучшие параметры для логистической регрессии: {grid_search.best_params_}")

Обучим на датасете random undersampling + smote

In [53]:
#используем функцию, которую написали ранее для того чтобы обучить логистическую регрессию на датасете random undersampling + smote
#grid_model_smote_rus = train_LogReg(X_resampled_smote_rus, y_resampled_smote_rus)

D:\conda\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [57]:
#y_pred = grid_model_smote_rus.best_estimator_.predict(X_test)

In [85]:
#print(f"Лучший roc_auc: {grid_model_smote_rus.best_score_:.4f}")

Лучший roc_auc: 0.9845


In [55]:
#print(confusion_matrix(y_test, y_pred))
#print(classification_report(y_test, y_pred))

[[98659  4031]
 [  121  3754]]
              precision    recall  f1-score   support

           0       1.00      0.96      0.98    102690
           1       0.48      0.97      0.64      3875

    accuracy                           0.96    106565
   macro avg       0.74      0.96      0.81    106565
weighted avg       0.98      0.96      0.97    106565



In [135]:
import pickle
with open('grid_model_smote_rus.pkl', 'wb') as file:
    pickle.dump(grid_model_smote_rus, file)

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [117]:
#используем функцию c другими параметрами, чтобы обучить на этом том же датасете + используем стандартизацию
scaler = StandardScaler()
grid_model_smote_rus_L1regularization = train_LogReg(scaler.fit_transform(X_resampled_smote_rus), (y_resampled_smote_rus), C=[0.001, 0.01, 0.1, 1, 10, 100], 
                 penalty=['l1'], solver=['liblinear'], max_iter=[100, 200, 300, 400],
                 scoring = 'f1')

In [118]:
y_pred = grid_model_smote_rus_L1regularization.best_estimator_.predict(scaler.transform(X_test))

In [119]:
print(f"Лучший f1: {grid_model_smote_rus_L1regularization.best_score_:.4f}")

Лучший f1: 0.9937


In [120]:
print(confusion_matrix(y_test, y_pred))#лучшая модель
print(classification_report(y_test, y_pred))

[[102106    584]
 [    35   3840]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00    102690
           1       0.87      0.99      0.93      3875

    accuracy                           0.99    106565
   macro avg       0.93      0.99      0.96    106565
weighted avg       0.99      0.99      0.99    106565



In [133]:
import pickle
with open('grid_model_smote_rus_l12_saga.pkl', 'wb') as file:
    pickle.dump(grid_model_smote_rus_L1regularization, file)

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [121]:
grid_model_smote_rus_l12_saga = train_LogReg(scaler.fit_transform(X_resampled_smote_rus), y_resampled_smote_rus, C=[0.001, 0.01, 0.1, 1, 10, 100], 
                 penalty=['l1', 'l2'], solver=['saga'], max_iter=[600, 400],
                 scoring = 'f1')

In [122]:
y_pred = grid_model_smote_rus_l12_saga.best_estimator_.predict(scaler.transform(X_test))

In [123]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[102109    581]
 [    34   3841]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00    102690
           1       0.87      0.99      0.93      3875

    accuracy                           0.99    106565
   macro avg       0.93      0.99      0.96    106565
weighted avg       0.99      0.99      0.99    106565



In [ ]:
with open('grid_model_smote_rus_L1regularization.pkl', 'wb') as file:
    pickle.dump(grid_model_smote_rus_L1regularization, file)

Обучим на датасете random Cluster Centroids, c метрикой roc_auc

In [73]:
grid_model_cc_L2regularization = train_LogReg(X_resampled_cc, y_resampled_cc, C=[0.001, 0.01, 0.1, 1, 10, 25], 
                 penalty=['l2'], solver=['lbfgs'], max_iter=[100, 200, 300, 400],
                 scoring = 'roc_auc')

D:\conda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [74]:
y_pred = grid_model_cc_L2regularization.best_estimator_.predict(X_test)

In [75]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[61381 41309]
 [  710  3165]]
              precision    recall  f1-score   support

           0       0.99      0.60      0.75    102690
           1       0.07      0.82      0.13      3875

    accuracy                           0.61    106565
   macro avg       0.53      0.71      0.44    106565
weighted avg       0.96      0.61      0.72    106565



теперь с метрикой f1

In [78]:
grid_model_cc_L2regularization_f1 = train_LogReg(X_resampled_cc, y_resampled_cc, C=[0.001, 0.01, 0.1, 1, 10, 25], 
                 penalty=['l2'], solver=['lbfgs'], max_iter=[300, 400, 500],
                 scoring = 'f1')

D:\conda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [79]:
y_pred = grid_model_cc_L2regularization_f1.best_estimator_.predict(X_test)

In [80]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[61716 40974]
 [  698  3177]]
              precision    recall  f1-score   support

           0       0.99      0.60      0.75    102690
           1       0.07      0.82      0.13      3875

    accuracy                           0.61    106565
   macro avg       0.53      0.71      0.44    106565
weighted avg       0.96      0.61      0.73    106565



Обучим на датасете SMOTEENN

In [81]:
grid_model_SMOTEENN_L12regularization = train_LogReg(X_resampled_cc, y_resampled_cc, C=[0.001, 0.01, 0.1, 1, 10, 25], 
                 penalty=['l1', 'l2'], solver=['saga'], max_iter=[100, 200, 300, 400],
                 scoring = 'roc_auc')

D:\conda\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [82]:
y_pred = grid_model_SMOTEENN_L12regularization.best_estimator_.predict(X_test)

In [83]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[99479  3211]
 [  364  3511]]
              precision    recall  f1-score   support

           0       1.00      0.97      0.98    102690
           1       0.52      0.91      0.66      3875

    accuracy                           0.97    106565
   macro avg       0.76      0.94      0.82    106565
weighted avg       0.98      0.97      0.97    106565



In [134]:
import pickle
with open('grid_model_SMOTEENN_L12regularization.pkl', 'wb') as file:
    pickle.dump(grid_model_SMOTEENN_L12regularization, file)

In [124]:
#проверим по f1, применяя elasticnet регуляризацию + standartscaler
grid_model_SMOTEENN_L12regularization_f1 = train_LogReg(scaler.fit_transform(X_resampled_cc), y_resampled_cc, C=[0.001, 0.01, 0.1, 1, 25], 
                 penalty=['elasticnet'], l1_ratio= [0.1, 0.5, 0.9, 1], solver=['saga'], max_iter=[600, 800],
                 scoring = 'f1')#но на самом деле elasticnet

In [125]:
y_pred = grid_model_SMOTEENN_L12regularization_f1.best_estimator_.predict(scaler.transform(X_test))

In [126]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[89727 12963]
 [   21  3854]]
              precision    recall  f1-score   support

           0       1.00      0.87      0.93    102690
           1       0.23      0.99      0.37      3875

    accuracy                           0.88    106565
   macro avg       0.61      0.93      0.65    106565
weighted avg       0.97      0.88      0.91    106565



Лучший результат показала модель "grid_model_smote_rus_L1regularization", с учётом, что данные подавались на вход стандартизованные. Стандартизация данных в некоторых случаях приводила к ухудшениям результатов.

Перейдём к методу RandomForest, работа с этим методом выстроена в том же порядке, как и для логистической регрессии

In [130]:
def train_RandomForest(X_resampled, y_resampled, n_estimators=[50, 100, 200],
                       max_features=['auto', 'sqrt', 'log2'], 
                       max_depth=[10, 20, 30, 40, 50], 
                       min_samples_split=[2, 5, 10], 
                       min_samples_leaf=[1, 2, 4], 
                       scoring='f1'):
    """
    Обучает модель случайного леса с использованием GridSearchCV для подбора гиперпараметров.

    Параметры:
    -----------
    X_resampled : array-like, shape (n_samples, n_features)
        Обучающие данные после балансировки (признаки).
        
    y_resampled : array-like, shape (n_samples,)
        Целевые переменные после балансировки (метки классов).
        
    n_estimators : list, optional (default=[50, 100, 200])
        Количество деревьев в случайном лесе. Большее количество деревьев может улучшить 
        производительность, но также увеличивает время обучения.
        
    max_features : list, optional (default=['auto', 'sqrt', 'log2'])
        Максимальное количество признаков, которые будут рассматриваться при поиске 
        наилучшего разделения. Меньшее количество может снизить переобучение.
        
    max_depth : list, optional (default=[None, 10, 20, 30, 40, 50])
        Максимальная глубина дерева. Ограничение глубины дерева может помочь 
        предотвратить переобучение.
        
    min_samples_split : list, optional (default=[2, 5, 10])
        Минимальное количество образцов, необходимых для разделения узла.
        
    min_samples_leaf : list, optional (default=[1, 2, 4])
        Минимальное количество образцов, необходимых для создания листа.
        
    scoring : str, optional (default='roc_auc')
        Метрика, используемая для оценки модели. Поддерживаемые значения включают 
        'roc_auc', 'accuracy', 'f1', и другие. Это значение будет использоваться 
        в качестве критерия для подбора гиперпараметров.

    Возвращаемое значение:
    -----------------------
    grid_search : GridSearchCV
        Объект GridSearchCV, содержащий информацию о лучших параметрах и оценках 
        модели.
    """
    
    # Создаем сетку гиперпараметров
    param_grid = {
        'n_estimators': n_estimators,
        'max_features': max_features,
        'max_depth': max_depth,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf
    }
    
    # Создаем экземпляр GridSearch
    grid_search = GridSearchCV(RandomForestClassifier(random_state=42), 
                               param_grid, 
                               cv=5, 
                               scoring=scoring, 
                               n_jobs=-1)  # Используем n_jobs=-1 для параллельного выполнения
    
    # Обучение
    grid_search.fit(X_resampled, y_resampled)

    # Возвращаем модель
    return grid_search

In [131]:
# Обучаем модель случайного леса на сбалансированной выборке, "победившей" в логистической регрессии
grid_model_rf = train_RandomForest(scaler.fit_transform(X_resampled_smote_rus), (y_resampled_smote_rus))

MemoryError: Unable to allocate 2.51 MiB for an array with shape (328606,) and data type int64

In [132]:
y_pred = grid_model_rf.predict(scaler.fit(X_test))#смотрим на результаты

NameError: name 'grid_model_rf' is not defined

In [ ]:
#метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')  # Для бинарной классификации
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')
roc_auc = roc_auc_score(y_test, grid_model_rf.predict_proba(X_test)[:, 1])

# Печатаем метрики
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

# Дополнительная информация
print("\nClassification Report:\n", classification_report(y_tes

Теперь подберём гиперпараметры для модели XGBoost

In [ ]:
def train_XGBoost(X_resampled, y_resampled, n_estimators=[100, 200, 300],
                  max_depth=[3, 5, 7], learning_rate=[0.01, 0.1, 0.2],
                  scoring='roc_auc'):
    """
    Обучает модель XGBoost с использованием GridSearchCV для подбора гиперпараметров.
    """
    param_grid = {
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'learning_rate': learning_rate
    }
    
    # Создаем экземпляр GridSearch
    grid_search = GridSearchCV(XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'), 
                               param_grid, cv=5, scoring=scoring, n_jobs=-1)
    
    # Обучение
    grid_search.fit(X_resampled, y_resampled)
    
    return grid_search

In [ ]:
grid_model_xgb = train_XGBoost(X_resampled_smote, y_resampled_smote)

In [ ]:
y_pred = grid_model_xgb.predict(X_test)

In [ ]:
# Вычисляем различные метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')  # Для бинарной классификации
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')
roc_auc = roc_auc_score(y_test, grid_model_xgb.predict_proba(X_test)[:, 1])

# Печатаем метрики
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

# Дополнительная информация
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Голосование!!!

In [ ]:
 voting_clf = VotingClassifier(estimators=models, voting='soft')  # Используем soft voting для вероятностей

In [ ]:
voting_clf.fit(X_train, y_train)#обучение ансамбля

In [ ]:
# Предсказание на тестовых данных
y_pred = voting_clf.predict(X_test)

In [ ]:
print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")

    # Дополнительная информация
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
def train_voting_classifier(models, X_train, y_train, X_test, y_test):
    """
    Обучает VotingClassifier с использованием предоставленных моделей и выводит метрики.

    Параметры:
    -----------
    models : list
        Список кортежей с моделями и их именами (например, [("log_reg", model1), ("rf", model2), ...]).
    
    X_train : array-like, shape (n_samples, n_features)
        Обучающие данные.
    
    y_train : array-like, shape (n_samples,)
        Целевые переменные для обучающих данных.
    
    X_test : array-like, shape (n_samples, n_features)
        Тестовые данные.
    
    y_test : array-like, shape (n_samples,)
        Целевые переменные для тестовых данных.

    Возвращаемое значение:
    -----------------------
    voting_clf : VotingClassifier
        Объект VotingClassifier, содержащий обученную модель.
    """
    
    # Создаем VotingClassifier
    voting_clf = VotingClassifier(estimators=models, voting='soft')  # Используем soft voting для вероятностей

    # Обучение
    voting_clf.fit(X_train, y_train)

    # Предсказание на тестовых данных
    y_pred = voting_clf.predict(X_test)

    # Вычисляем и выводим метрики
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='binary')  # Для бинарной классификации
    recall = recall_score(y_test, y_pred, average='binary')
    f1 = f1_score(y_test, y_pred, average='binary')
    roc_auc = roc_auc_score(y_test, voting_clf.predict_proba(X_test)[:, 1])

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")

    # Дополнительная информация
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

    return voting_clf

In [ ]:
#Нужно передать лучшие модели по итогам подбора гиперпараметров
log_reg_model = grid_model_log_reg.best_estimator_  # Лучшая модель логистической регрессии
rf_model = grid_model_rf.best_estimator_  # Лучшая модель случайного леса
xgb_model = grid_model_xgb.best_estimator_  # Лучшая модель XGBoost

# Список моделей для VotingClassifier
models = [
    ('log_reg', log_reg_model),
    ('rf', rf_model),
    ('xgb', xgb_model)
]

In [ ]:
# обучение VotingClassifier
voting_model = train_voting_classifier(models, X_resampled_smote_rus, y_resampled_smote_rus, X_test, y_test)

Функция, которая автоматически проверит все созданные модели, и выведет результаты.

In [ ]:
test = pd.read_csv()

In [ ]:
model_name = []
model_ = []

for m in zip(model_name, model): 
    print(m[0], f1_score)